# Block2SimMC Demonstration

In [1]:
from PSSimPy import Account, Transaction
from PSSimPy.queues import PriorityQueue
from textwrap import dedent
from pyModelChecking import Kripke
import pyModelChecking.LTL as LTL   
from itertools import product
from abc import ABC, abstractmethod
from dataclasses import dataclass, asdict

from simulator_class import CollateralizedCreditFacility
from modules import ScopeDefinition, PropertyExtraction, AdapterBounding

# Scope Definition

In [2]:
contract_path = 'smart_contract/priority_queue.sol'
scope_definition = ScopeDefinition(contract_path)
scope = scope_definition.extract_scope()

# Property Extraction

In [3]:
prop_extractor = PropertyExtraction(scope, "config/properties.json")
properties = prop_extractor.extract_properties()

In [4]:
properties

{'safety': [('NoDoubleDequeue', 'G( !(dequeue(tx) && dequeue(tx)) )'),
  ('PriorityRespected',
   'G( enqueue(tx1) && enqueue(tx2) && priority(tx2) < priority(tx1) -> ! (dequeue(tx1) before dequeue(tx2)) )'),
  ('FIFOOnTie',
   'G( samePriority(tx1,tx2) && earlierDayTime(tx1,tx2) -> ! (dequeue(tx2) before dequeue(tx1)) )')],
 'liveness': [('EnqueueLeadsToDequeue', 'G( enqueue(tx) -> F dequeue(tx) )')]}

# Adapter & Bounding

In [ ]:
# 1. Prepare accounts and any initial token/state setup
accounts = [Account(id='acc1', owner=None, balance=5), Account(id='acc2', owner=None, balance=10)]
account_by_id = {acc.id: acc for acc in accounts}

# --- (2) Define snapshot_fn and load_fn ---
def pq_snapshot(pq: PriorityQueue):
    # 1. Serialize the queue as tuples of primitives
    entries = []
    for txn, period in pq.queue:
        entries.append((
            txn.sender_account.id,
            txn.amount,
            txn.priority,
            txn.day,
            txn.time,
            period
        ))
    # 2. Also capture the account balances
    balances = tuple(acc.balance for acc in accounts)
    # Return a fully hashable state
    return (tuple(entries), balances)

def pq_load(pq: PriorityQueue, state):
    entries, balances = state
    # 1. Reset the queue
    pq.queue.clear()
    # 2. Reconstruct each Transaction and re-add
    for sender_id, amount, prio, day, time, period in entries:
        acct = account_by_id[sender_id]
        txn  = Transaction(
            sender_account=acct,
            amount=amount,
            priority=prio,
            day=day,
            time=time
        )
        # We bypass any balance check in enqueue; instead, insert directly
        pq.queue.add((txn, period))
    # 3. Restore balances
    for acc, bal in zip(accounts, balances):
        acc.balance = bal

# --- (3) Define actions and bounds ---
actions = [
    {
        "name":   "enqueue",
        "params": [(1,0,1,"08:00"), (2,1,2,"09:00")],
        "label":  "enqueue",
        "guard":  lambda inst, args: True
    },
    {
        "name":   "dequeue",
        "params": [()],
        "label":  "dequeue",
        "guard":  lambda inst, _: True
    }
]

bounds = {
    "amounts":    [1,2],
    "priorities": [0,1],
    "max_day":     2,
    "times":      ["08:00","09:00"]
}

# --- (4) Instantiate the generic Adapter & Bounding ---
adapter = AdapterBounding(
    impl_cls         = PriorityQueue,
    constructor_args = (),
    constructor_kwargs={},
    actions          = actions,
    snapshot_fn      = pq_snapshot,
    load_fn          = pq_load,
    bounds           = bounds
)

# --- (5) Test ---
init_states = list(adapter.initial_states())
print("Initial state:", init_states)

succs = list(adapter.successors(init_states[0]))
print("Some successors:", succs[:3])

Initial state: [State(data=((), (5, 10)))]
Some successors: []
